# Bag of Words Meets A Bag of Popcorns

## Word Vectors

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("labeledTrainData.tsv", header = 0, delimiter= "\t", quoting=3)

In [3]:
test = pd.read_csv("testData.tsv", header = 0, delimiter="\t", quoting=3)

In [4]:
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header = 0, delimiter="\t", quoting=3)

In [5]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews" \
      %(train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews


## Data cleaning and processing

In [7]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [8]:
def review_to_wordlist(review, remove_stopwords = False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

In [9]:
import nltk.data

In [10]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [11]:
def review_to_sentences(review, tokenizer, remove_stopwords = False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist( raw_sentence, remove_stopwords))
        return sentences

In [12]:
# Download the punkt tokenizer for sentence splitting
import nltk.data

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [13]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\hongji\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\hongji\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\hongji\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


C:\Users\hongji\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\hongji\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\hongji\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful S

In [14]:
print(len(sentences))

795538


In [15]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [16]:
print(sentences[1])

['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [17]:
import logging
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [18]:
num_features = 300
min_word_count = 40
num_workers = 4
context =10
downsampling = 1e-3

from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers = num_workers, size = num_features, min_count = min_word_count, window = context, sample = downsampling)


C:\Users\hongji\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-12-07 16:22:57,910 : INFO : 'pattern' package not found; tag filters are not available for English
2018-12-07 16:22:57,916 : INFO : collecting all words and their counts
2018-12-07 16:22:57,916 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-07 16:22:57,956 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2018-12-07 16:22:57,996 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2018-12-07 16:22:58,035 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2018-12-07 16:22:58,076 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2018-12-07 16:22:58,120 : INFO : PROGRESS: at sentence #5000

Training model...


2018-12-07 16:22:58,159 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2018-12-07 16:22:58,200 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2018-12-07 16:22:58,244 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2018-12-07 16:22:58,286 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2018-12-07 16:22:58,327 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2018-12-07 16:22:58,372 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2018-12-07 16:22:58,412 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keeping 54119 word types
2018-12-07 16:22:58,453 : INFO : PROGRESS: at sentence #130000, processed 2894303 words, keeping 55847 word types
2018-12-07 16:22:58,505 : INFO : PROGRESS: at sentence #140000, processed 3107005 words, kee

2018-12-07 16:23:01,402 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2018-12-07 16:23:01,450 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2018-12-07 16:23:01,476 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2018-12-07 16:23:01,477 : INFO : Loading a fresh vocabulary
2018-12-07 16:23:01,541 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2018-12-07 16:23:01,541 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2018-12-07 16:23:01,591 : INFO : deleting the raw counts dictionary of 123504 items
2018-12-07 16:23:01,595 : INFO : sample=0.001 downsamples 48 most-common words
2018-12-07 16:23:01,596 : INFO : downsampling leaves estimated 12749658 word corpus (74.0% of prior 17238940)
2018-12-07 16:23:01,648 : INFO : estimated required memory for 16490 words and 300 dimensi

2018-12-07 16:23:53,531 : INFO : EPOCH 4 - PROGRESS: at 69.19% examples, 878244 words/s, in_qsize 7, out_qsize 0
2018-12-07 16:23:54,531 : INFO : EPOCH 4 - PROGRESS: at 76.27% examples, 880294 words/s, in_qsize 7, out_qsize 0
2018-12-07 16:23:55,532 : INFO : EPOCH 4 - PROGRESS: at 83.39% examples, 882583 words/s, in_qsize 7, out_qsize 0
2018-12-07 16:23:56,541 : INFO : EPOCH 4 - PROGRESS: at 90.65% examples, 885511 words/s, in_qsize 7, out_qsize 0
2018-12-07 16:23:57,541 : INFO : EPOCH 4 - PROGRESS: at 97.78% examples, 887147 words/s, in_qsize 6, out_qsize 1
2018-12-07 16:23:57,823 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-12-07 16:23:57,830 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-12-07 16:23:57,837 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-12-07 16:23:57,843 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-12-07 16:23:57,844 : INFO : EPOCH - 4 : training on 17798082 raw w

In [19]:
model.init_sims(replace = True)

2018-12-07 16:24:12,677 : INFO : precomputing L2-norms of word weight vectors


In [20]:
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-12-07 16:24:12,872 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2018-12-07 16:24:12,874 : INFO : not storing attribute vectors_norm
2018-12-07 16:24:12,874 : INFO : not storing attribute cum_table
2018-12-07 16:24:13,343 : INFO : saved 300features_40minwords_10context


### Exploring the Model Results

In [21]:
model.doesnt_match("man woman child kitchen".split())

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'kitchen'

In [22]:
model.doesnt_match("france england germany berlin".split())

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'berlin'

In [23]:
model.doesnt_match("paris berlin london austria".split())

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'paris'

In [24]:
model.most_similar("man")

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6199898719787598),
 ('lady', 0.5875682830810547),
 ('monk', 0.541907787322998),
 ('guy', 0.540055513381958),
 ('lad', 0.5359101891517639),
 ('person', 0.5129542350769043),
 ('farmer', 0.5112081170082092),
 ('millionaire', 0.5098932981491089),
 ('poet', 0.5077937245368958),
 ('soldier', 0.5071138739585876)]

In [25]:
model.most_similar("queen")

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6501305103302002),
 ('bride', 0.6225188374519348),
 ('belle', 0.6057083606719971),
 ('maria', 0.5866658091545105),
 ('showgirl', 0.5830126404762268),
 ('voluptuous', 0.5761037468910217),
 ('mistress', 0.5753309726715088),
 ('stepmother', 0.5740993618965149),
 ('regina', 0.5702788829803467),
 ('goddess', 0.5691624879837036)]

In [26]:
model.wv.most_similar("queen")

[('princess', 0.6501305103302002),
 ('bride', 0.6225188374519348),
 ('belle', 0.6057083606719971),
 ('maria', 0.5866658091545105),
 ('showgirl', 0.5830126404762268),
 ('voluptuous', 0.5761037468910217),
 ('mistress', 0.5753309726715088),
 ('stepmother', 0.5740993618965149),
 ('regina', 0.5702788829803467),
 ('goddess', 0.5691624879837036)]

In [27]:
model.wv.most_similar("awful")

[('terrible', 0.7752892374992371),
 ('horrible', 0.7612673044204712),
 ('atrocious', 0.7319850921630859),
 ('dreadful', 0.7110860347747803),
 ('abysmal', 0.7070614099502563),
 ('appalling', 0.6937450766563416),
 ('horrendous', 0.690858006477356),
 ('horrid', 0.6669932007789612),
 ('lousy', 0.6175165176391602),
 ('crappy', 0.6162477731704712)]

### Numeric Representations of Words

In [28]:
from gensim.models import Word2Vec

model = Word2Vec.load("300features_40minwords_10context")

2018-12-07 16:24:13,420 : INFO : loading Word2Vec object from 300features_40minwords_10context
2018-12-07 16:24:13,764 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2018-12-07 16:24:13,765 : INFO : setting ignored attribute vectors_norm to None
2018-12-07 16:24:13,765 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2018-12-07 16:24:13,766 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2018-12-07 16:24:13,766 : INFO : setting ignored attribute cum_table to None
2018-12-07 16:24:13,767 : INFO : loaded 300features_40minwords_10context


In [29]:
type(model.wv.syn0)

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [30]:
model.wv.syn0.shape

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(16490, 300)

In [31]:
model["flower"]

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-6.79905489e-02,  4.79103401e-02, -7.51963407e-02, -7.59368315e-02,
        3.18425559e-02,  7.46567100e-02,  1.43892169e-02, -1.16791233e-01,
        1.13090694e-01,  5.09426482e-02,  4.31687124e-02,  1.25429593e-03,
       -8.40775073e-02,  9.23774838e-02,  8.08008909e-02,  1.19474724e-01,
       -9.90575701e-02,  1.29886687e-01,  2.33578049e-02, -7.16446294e-03,
       -1.11426321e-04,  8.59347284e-02,  2.12857220e-02,  6.49599871e-03,
       -2.37856377e-02,  1.19387291e-01,  6.49650395e-02,  2.89626569e-02,
       -2.72357860e-03,  4.10408154e-02, -3.72854657e-02, -9.46135893e-02,
        8.01337063e-02, -9.58624333e-02, -1.21818595e-01, -1.12912841e-02,
        5.71373552e-02, -1.66012831e-02, -2.96142977e-02, -8.85493830e-02,
        5.05695306e-02,  6.40890673e-02,  3.42576727e-02, -3.62741500e-02,
        9.64733511e-02,  5.63212372e-02, -6.51532784e-02,  2.83266827e-02,
        9.77551751e-03, -4.12805490e-02, -3.16814194e-03, -5.11106364e-02,
        5.10611795e-02,  

### From Words To Paragraphs, Attempt 1: Vector Averaging

In [32]:
import numpy as np

In [33]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype = "float32")
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords = nwords +1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [34]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype = "float32")
    
    for review in reviews:
        if counter%1000. == 0.:
            print("Review %d of %d" % (counter, len(reviews)))
        
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        
        counter = counter + 1
    return reviewFeatureVecs

In [35]:
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords = True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000


C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [36]:
trainDataVecs

array([[-1.22751715e-02, -2.28399653e-02,  1.54675404e-02, ...,
         3.55393649e-03,  1.02672046e-02, -1.08140509e-03],
       [-4.02781516e-02, -2.21547037e-02,  4.35531884e-02, ...,
         1.58176292e-02,  1.13732219e-02,  6.55760244e-03],
       [-2.17449795e-02, -3.47934058e-03,  2.20970642e-02, ...,
         2.82143810e-05, -8.82775057e-03,  4.53439448e-03],
       ...,
       [-4.74437140e-03, -3.02658062e-02,  2.76851412e-02, ...,
         7.74862361e-04,  8.89352243e-03,  6.67165639e-03],
       [-3.38495634e-02, -4.08084430e-02,  2.63757557e-02, ...,
         7.25536491e-04, -1.94670241e-02, -4.95104166e-03],
       [-4.29469347e-02, -2.60304827e-02,  2.50425301e-02, ...,
         2.11889227e-03, -1.64426398e-02,  6.92247692e-03]], dtype=float32)

In [37]:
testDataVecs

array([[-0.02519491, -0.03208986,  0.02594041, ..., -0.01933474,
        -0.01815491, -0.00064313],
       [-0.02925995, -0.02932343,  0.04511272, ..., -0.00550658,
         0.00380823,  0.01080793],
       [-0.03003048, -0.04275   ,  0.04341628, ..., -0.00849386,
         0.00066014,  0.00673819],
       ...,
       [-0.02959181, -0.02622495,  0.02530962, ..., -0.00733268,
         0.01838987,  0.00809767],
       [-0.02866081, -0.01544304, -0.00133891, ..., -0.01768848,
        -0.00772899,  0.00056236],
       [-0.02626836, -0.02471805,  0.02522402, ...,  0.00470375,
        -0.00574452,  0.00684188]], dtype=float32)

In [38]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainDataVecs, train["sentiment"])

result = forest.predict(testDataVecs)

output = pd.DataFrame(data = {"id":test["id"], "sentiment":result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting = 3)

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Fitting a random forest to labeled training data...


### From Words to Paragraphs, Attempt 2: Clustering

In [39]:
from sklearn.cluster import KMeans
import time

In [40]:
start = time.time()

model.wv.syn0

word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0] / 5)

kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\hongji\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """
C:\Users\hongji\Continuum\anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


Time taken for K Means clustering:  580.6943757534027 seconds.


In [41]:
word_centroid_map = dict(zip(model.wv.index2word, idx))

In [52]:
for cluster in range(0,10):
    print("\nCluster %d" % cluster)
    words = []
    for i in range(0, len(word_centroid_map.values())):
        if(list(word_centroid_map.values())[i] == cluster):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['boss', 'leader', 'rival', 'pirate', 'rivals', 'arch', 'foe', 'recruit']

Cluster 1
['sophisticated', 'refined']

Cluster 2
['terry', 'doc', 'benjamin', 'noah', 'sherman', 'clyde', 'gilliam', 'kramer', 'cromwell', 'theodore', 'mckenzie', 'arabia', 'terrence', 'forsyth', 'aldrich', 'flaherty', 'costa', 'koch']

Cluster 3
['disappear', 'vanish']

Cluster 4
['smash', 'plate', 'ropes', 'bolt']

Cluster 5
['intelligence', 'decency', 'intellect']

Cluster 6
['mainland', 'beijing']

Cluster 7
['overkill', 'repetitious']

Cluster 8
['remarkable', 'magnificent', 'fabulous', 'marvelous', 'tremendous', 'splendid', 'masterful', 'sublime', 'shakespearean', 'superlative', 'sparkling', 'robust', 'sturdy', 'bravura']

Cluster 9
['fun', 'entertaining', 'enjoyable', 'exciting', 'amusing', 'watchable', 'addictive', 'recommendable']


In [58]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    num_centroids = max(word_centroid_map.values()) + 1
    bag_of_centroids = np.zeros(num_centroids, dtype="float32")
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [59]:
train_centroids = np.zeros((train["review"].size, num_clusters), dtype = "float32")

counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1
    
test_centroids = np.zeros((test["review"].size, num_clusters), dtype = "float32")

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1
    


In [60]:
forest = RandomForestClassifier(n_estimators=100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids, train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data = {"id":test["id"], "sentiment": result})
output.to_csv("BagOfCentroids.csv", index = False, quoting= 3)

Fitting a random forest to labeled training data...
